In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import numpy as np
import tensorflow as tf
import functools
from matplotlib import pyplot as plt
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="1"

from gantools import utils, plot, data, evaluation, blocks
from gantools.gansystem import GANsystem, UpcaleGANsystem
from gantools.model import WGAN, UpscalePatchWGAN, CosmoWGAN
from functools import partial
 


In [ ]:
# Setting the forward and backward transform

# forward = data.fmap.log_forward
# backward = data.fmap.log_backward

forward = partial(data.fmap.stat_forward, c=20000, shift=3)
backward = partial(data.fmap.stat_backward, c=20000, shift=3)


# number of sample generated
num_samples = 30

pathmodel = '../saved_results/nbody'
pathsample = '../samples/nbody'

In [ ]:
def plot_cubes_paper(cubes, slice_num=None, nx=2, ny=2, bkd=backward, **kwargs):
    def map_to_plot(img):
        return data.fmap.log_forward(bkd(img))
    if len(cubes)<nx*ny:
        raise ValueError("Not enough samples.")
    if slice_num is None:
        slice_num = cubes.shape[1]//2+4
    fig, ax = plt.subplots(nx, ny, sharey=True,figsize=(11/2*nx,10.5/2*ny))
    sn = 0
    for i in range(nx):
        for j in range(ny):
            if nx==1 and ny==1:
                tax = ax
            elif nx==1:
                tax = ax[j]
            elif ny==1:
                tax = ax[i]
            else:
                tax = ax[i,j]
            tax.imshow(map_to_plot(cubes[sn,slice_num]), interpolation='none', **kwargs)
            tax.axis('off')
            sn += 1
    plt.tight_layout()

    return fig

# Generate samples - 32->64->256

## 0->32

In [ ]:
name = 'WGAN_stat2-0_to_32_checkpoints'
checkpoint32 = 44000

pathmodel32 = os.path.join(pathmodel, name)
params_32 = utils.load_params(pathmodel32)
# params_32 = old2new_params(params_32, CosmoWGAN)
# params_32['net']['shape'] = [*params_32['net']['shape'],1]

# params_32['net']['cosmology']['backward_map'] = backward
# params_32['net']['cosmology']['forward_map'] = forward
# params_32['summary_dir'] = os.path.join(pathmodel, name +'summary/')

# for ind, item in enumerate(params.items()):
#     print(ind, " : ", item[0], " - ", item[1])
# print(resume)


In [ ]:
wgan_32 = GANsystem(CosmoWGAN,params_32)

In [ ]:
gen_samples_32 = wgan_32.generate(N=num_samples, checkpoint=checkpoint32)
gen_samples_32 = np.squeeze(gen_samples_32)

In [ ]:
os.makedirs(os.path.join(pathsample,'stat2-0_32/'), exist_ok=True)
utils.save_hdf5(gen_samples_32, os.path.join(pathsample,'stat2-0_32/32_samples_ckpt_latest.h5'))

In [ ]:
cmin = 0.1
cmax = 2
clim = (cmin, cmax)
plot_cubes_paper(gen_samples_32, cmap=plt.cm.plasma, clim=clim, nx=4, ny=4);

## 32->64

Load fake 32

In [ ]:
gen_samples_32 = utils.load_hdf5(os.path.join(pathsample,'stat2-0_32/32_samples_ckpt_latest.h5'))[:num_samples]
print("gen_samples_32 shape=", gen_samples_32.shape)
gen_samples_32 = np.expand_dims(gen_samples_32, axis=4)
print('downsampled shape=', gen_samples_32.shape)

In [ ]:
name = 'WGAN_stat2-32_to_64_checkpoints'
checkpoint64 = 201000 #160000 #70000
class CosmoUpscalePatchWGAN(UpscalePatchWGAN, CosmoWGAN):
    pass

pathmodel64 = os.path.join(pathmodel, name)
params_64 = utils.load_params(pathmodel64)

In [ ]:
wgan_64 = UpcaleGANsystem(CosmoUpscalePatchWGAN, params_64)
gen_samples_64 = wgan_64.upscale_image(small=gen_samples_32, checkpoint=checkpoint64)

In [ ]:
os.makedirs(os.path.join(pathsample,'stat2-32_64/'), exist_ok=True)
utils.save_hdf5(gen_samples_64, os.path.join(pathsample,'stat2-32_64/64_samples_ckpt_latest.h5'))

In [ ]:
cmin = 0
cmax = 2.5
clim = (cmin, cmax)
plot_cubes_paper(gen_samples_64, cmap=plt.cm.plasma, clim=clim, nx=4, ny=4);


## 64->256

In [ ]:
gen_samples_64 = utils.load_hdf5(os.path.join(pathsample,'stat2-32_64/64_samples_ckpt_latest.h5'))

print('gen_samples_64 shape=', gen_samples_64.shape)
gen_samples_64 = np.expand_dims(gen_samples_64, axis=4)[:num_samples]
print('downsampled shape=', gen_samples_64.shape)

In [ ]:
name = 'WGAN_stat2-64_to_256_checkpoints'
checkpoint256 = 47000 #133000 #62000
# name = 'WGAN_stat-64_to_256_checkpoints'
# checkpoint256 = 130000 #191000
pathmodel256 = os.path.join(pathmodel, name)

params_256 = utils.load_params(pathmodel256)

In [ ]:
wgan_256 = UpcaleGANsystem(CosmoUpscalePatchWGAN, params_256)
gen_samples_256 = wgan_256.upscale_image(small=gen_samples_64, checkpoint=checkpoint256)

In [ ]:
cmin = 0
cmax = 3
clim = (cmin, cmax)
plot_cubes_paper(gen_samples_256, cmap=plt.cm.plasma, clim=clim, nx=2, ny=2);

In [ ]:
os.makedirs(os.path.join(pathsample,'stat2-64_256/'), exist_ok=True)
utils.save_hdf5(gen_samples_256, os.path.join(pathsample,'stat2-64_256/256_samples_ckpt_latest.h5'))

# Uniscale model
This is not working. There is a special branch for this model.

In [ ]:
name = 'WGAN_stat-uniscale_checkpoints'
checkpointuniscale = 275000 #109000 

pathmodel_uniscale = os.path.join(pathmodel, name)
params_uniscale = utils.load_params(pathmodel_uniscale)
class CosmoUpscalePatchWGAN(UpscalePatchWGAN, CosmoWGAN):
    pass
params_uniscale['upscaling']=None



In [ ]:
wgan_uniscale = UpcaleGANsystem(CosmoUpscalePatchWGAN, params_uniscale)
gen_samples_uniscale = wgan_uniscale.upscale_image(N=num_samples, resolution=256, checkpoint=checkpointuniscale)

In [ ]:
os.makedirs(os.path.join(pathsample,'stat2-uniscale/'), exist_ok=True)
utils.save_hdf5(gen_samples_uniscale, os.path.join(pathsample,'stat2-uniscale/256_samples_ckpt_latest.h5'))

In [ ]:
cmin = 0
cmax = 3
clim = (cmin, cmax)
plot_cubes_paper(gen_samples_uniscale, cmap=plt.cm.plasma, clim=clim, nx=2, ny=2);

# Make the figures for the paper

In [ ]:
pathfig = 'figures/'
os.makedirs(pathfig, exist_ok=True)

#### Load all samples

In [ ]:
gen_samples_32 = utils.load_hdf5(os.path.join(pathsample,'stat2-0_32/32_samples_ckpt_latest.h5'))
gen_samples_32_raw = backward(gen_samples_32)

In [ ]:
gen_samples_64 = utils.load_hdf5(os.path.join(pathsample,'stat2-32_64/64_samples_ckpt_latest.h5'))
gen_samples_64_raw = backward(gen_samples_64)

In [ ]:
gen_samples_256 = utils.load_hdf5(os.path.join(pathsample,'stat2-64_256/256_samples_ckpt_latest.h5'))
gen_samples_256_raw = backward(gen_samples_256)

In [ ]:
gen_samples_uniscale = utils.load_hdf5(os.path.join(pathsample,'stat2-uniscale/256_samples_ckpt_latest.h5'))
gen_samples_uniscale_raw = backward(gen_samples_uniscale)


#### Load real data

In [ ]:
dataset256 = data.load.load_nbody_dataset(resolution=256,Mpch=350,shuffle=False,forward_map=forward,spix=256,patch=False,is_3d=True,augmentation=False)
real_samples_256 = dataset256.get_all_data()
del dataset256


In [ ]:
real_samples_256_raw = backward(real_samples_256)

In [ ]:
dataset32 = data.load.load_nbody_dataset(resolution=256,Mpch=350,shuffle=False,forward_map=forward,spix=32,patch=False,is_3d=True,augmentation=False, scaling=8)
real_samples_32 = dataset32.get_all_data()
del dataset32

In [ ]:
real_samples_32_raw = backward(real_samples_32)

In [ ]:
dataset64 = data.load.load_nbody_dataset(resolution=256,Mpch=350,shuffle=False,forward_map=forward,spix=64,patch=False,is_3d=True,augmentation=False, scaling=4)
real_samples_64 = dataset64.get_all_data()
del dataset64

In [ ]:
real_samples_64_raw = backward(real_samples_64)

In [ ]:
# cmin = np.min(cubes)
# cmax = np.max(cubes)/1.5
cmin = 0
cmax = 3

clim = (cmin, cmax)
fig = plot_cubes_paper(real_samples_256, cmap=plt.cm.plasma, clim=clim);
fig.suptitle('Real $256^3$', y=1.02, fontsize=24 )
plt.savefig(pathfig+"real256.pdf", bbox_inches='tight', format='pdf')

fig2 = plot_cubes_paper(gen_samples_256, cmap=plt.cm.plasma, clim=clim);
fig2.suptitle('Fake $256^3$', y=1.02, fontsize=24 )
plt.savefig(pathfig+"fake256.pdf", bbox_inches='tight', format='pdf')



In [ ]:
logel2, l2, logel1, l1 = evaluation.compute_and_plot_mass_hist(real_samples_256_raw, gen_samples_256_raw)
plt.savefig(pathfig+"256full_hist.pdf", bbox_inches='tight', format='pdf')

In [ ]:
logel2, l2, logel1, l1 = evaluation.compute_and_plot_peak_cout(real_samples_256_raw, gen_samples_256_raw)
plt.savefig(pathfig+"256full_peak.pdf", bbox_inches='tight', format='pdf')

In [ ]:
evaluation.compute_and_plot_psd(real_samples_256_raw, gen_samples_256_raw)
plt.savefig(pathfig+"256full_psd.pdf", bbox_inches='tight', format='pdf')

# Scale by scale analysis

## 32 cubes

In [ ]:
# cmin = np.min(cubes_32)
# # cmin = 0
# cmax = np.max(cubes_32)
cmin = 0.1
cmax = 2
clim = (cmin, cmax)
fig = plot_cubes_paper(real_samples_32, cmap=plt.cm.plasma, clim=clim, nx=4, ny=4);
fig.suptitle('Real $32^3$', y=1.03, fontsize=48 )
plt.savefig(pathfig+"real32.pdf", bbox_inches='tight', format='pdf')

fig2 = plot_cubes_paper(gen_samples_32, cmap=plt.cm.plasma, clim=clim, nx=4, ny=4);
fig2.suptitle('Fake $32^3$', y=1.03, fontsize=48 )
plt.savefig(pathfig+"fake32.pdf", bbox_inches='tight', format='pdf')



In [ ]:
logel2, l2, logel1, l1 = evaluation.compute_and_plot_mass_hist(real_samples_32_raw, gen_samples_32_raw)
plt.savefig(pathfig+"32_hist.pdf", bbox_inches='tight', format='pdf')

In [ ]:
logel2, l2, logel1, l1 = evaluation.compute_and_plot_peak_cout(real_samples_32_raw, gen_samples_32_raw)
plt.savefig(pathfig+"32_peak.pdf", bbox_inches='tight', format='pdf')

In [ ]:
evaluation.compute_and_plot_psd(real_samples_32_raw, gen_samples_32_raw)
plt.savefig(pathfig+"32_psd.pdf", bbox_inches='tight', format='pdf')

## 64 Cubes

In [ ]:
name = 'WGAN_stat2-32_to_64_checkpoints'
class CosmoUpscalePatchWGAN(UpscalePatchWGAN, CosmoWGAN):
    pass
pathmodel64 = os.path.join(pathmodel, name)
params_64 = utils.load_params(pathmodel64)
wgan_64 = UpcaleGANsystem(CosmoUpscalePatchWGAN, params_64)
gen_samples_64_single = wgan_64.upscale_image(small=np.reshape(real_samples_32, [*real_samples_32.shape,1]), checkpoint=checkpoint64)

In [ ]:
gen_samples_64_single_raw = backward(gen_samples_64_single)

In [ ]:
# cmin = np.min(cubes_64)
# cmax = np.max(cubes_64)
cmin = 0
cmax = 2.5
clim = (cmin, cmax)
fig = plot_cubes_paper(real_samples_64, cmap=plt.cm.plasma, clim=clim);
fig.suptitle('Real $64^3$', y=1.04, fontsize=36 )
plt.savefig(pathfig+"up_real64.pdf", bbox_inches='tight', format='pdf')

fig2 = plot_cubes_paper(gen_samples_64_single, cmap=plt.cm.plasma, clim=clim);
fig2.suptitle('Fake $64^3$', y=1.04, fontsize=36 )
plt.savefig(pathfig+"up_fake64.pdf", bbox_inches='tight', format='pdf')

fig = plot_cubes_paper(real_samples_32, cmap=plt.cm.plasma, clim=clim);
fig.suptitle('Real downsampled $32^3$', y=1.04, fontsize=36 )
plt.savefig(pathfig+"up_down32.pdf", bbox_inches='tight', format='pdf')

In [ ]:
logel2, l2, logel1, l1 = evaluation.compute_and_plot_mass_hist(real_samples_64_raw, gen_samples_64_single_raw)
plt.savefig(pathfig+"up_64_hist.pdf", bbox_inches='tight', format='pdf')

In [ ]:
logel2, l2, logel1, l1 = evaluation.compute_and_plot_peak_cout(real_samples_64_raw, gen_samples_64_single_raw)
plt.savefig(pathfig+"up_64_peak.pdf", bbox_inches='tight', format='pdf')

In [ ]:
evaluation.compute_and_plot_psd(real_samples_64_raw, gen_samples_64_single_raw)
plt.savefig(pathfig+"up_64_psd.pdf", bbox_inches='tight', format='pdf')

## 256 Cubes

In [ ]:
name = 'WGAN_stat2-64_to_256_checkpoints'
pathmodel256 = os.path.join(pathmodel, name)

params_256 = utils.load_params(pathmodel256)

wgan_256 = UpcaleGANsystem(CosmoUpscalePatchWGAN, params_256)
gen_samples_single_256 = wgan_256.upscale_image(small=np.reshape(real_samples_64, [*real_samples_64.shape, 1]), checkpoint=checkpoint256)

In [ ]:
# cmin = np.min(cubes_256)
# cmax = np.max(cubes_256)/2.5
cmin = 0
cmax = 3
clim = (cmin, cmax)
fig = plot_cubes_paper(real_samples_256, cmap=plt.cm.plasma, clim=clim, ny=1);
fig.suptitle('Real $256^3$', y=1.03, fontsize=12 )
plt.savefig(pathfig+"up_real256.pdf", bbox_inches='tight', format='pdf')

fig2 = plot_cubes_paper(gen_samples_single_256, cmap=plt.cm.plasma, clim=clim, ny=1);
fig2.suptitle('Fake $256^3$', y=1.03, fontsize=12 )
plt.savefig(pathfig+"up_fake256.pdf", bbox_inches='tight', format='pdf')

fig = plot_cubes_paper(real_samples_64, cmap=plt.cm.plasma, clim=clim, ny=1);
fig.suptitle('Real downsampled $64^3$', y=1.03, fontsize=12 )
plt.savefig(pathfig+"up_down64.pdf", bbox_inches='tight', format='pdf')

In [ ]:
gen_samples_single_256_raw = backward(gen_samples_single_256)

In [ ]:
logel2, l2, logel1, l1 = evaluation.compute_and_plot_mass_hist(real_samples_256_raw, gen_samples_single_256_raw)
plt.savefig(pathfig+"up_256_hist.pdf", bbox_inches='tight', format='pdf')

In [ ]:
logel2, l2, logel1, l1 = evaluation.compute_and_plot_peak_cout(real_samples_256_raw, gen_samples_single_256_raw)
plt.savefig(pathfig+"up_256_peak.pdf", bbox_inches='tight', format='pdf')

In [ ]:
evaluation.compute_and_plot_psd(real_samples_256_raw, gen_samples_single_256_raw)
plt.savefig(pathfig+"up_256_psd.pdf", bbox_inches='tight', format='pdf')

In [ ]:
fig2 = plot_cubes_paper(gen_samples_single_256, cmap=plt.cm.plasma, clim=clim, ny=1, nx=1);
fig2 = plot_cubes_paper(real_samples_256, cmap=plt.cm.plasma, clim=clim, ny=1, nx=1);

## Uniscale

In [ ]:
cmin = 0
cmax = 3
clim = (cmin, cmax)
fig = plot_cubes_paper(real_samples_256, cmap=plt.cm.plasma, clim=clim);
fig.suptitle('Real $256^3$', y=1.02, fontsize=24 )
plt.savefig(pathfig+"uniscalereal256.pdf", bbox_inches='tight', format='pdf')

fig2 = plot_cubes_paper(gen_samples_uniscale, cmap=plt.cm.plasma, clim=clim);
fig2.suptitle('Fake $256^3 - uniscale$', y=1.02, fontsize=24 )
plt.savefig(pathfig+"uniscalefake256.pdf", bbox_inches='tight', format='pdf')

In [ ]:
logel2, l2, logel1, l1 = evaluation.compute_and_plot_mass_hist(real_samples_256_raw, gen_samples_uniscale_raw)
plt.savefig(pathfig+"256uniscale_hist.pdf", bbox_inches='tight', format='pdf')

In [ ]:
logel2, l2, logel1, l1 = evaluation.compute_and_plot_peak_cout(real_samples_256_raw, gen_samples_uniscale_raw)
plt.savefig(pathfig+"256uniscale_peak.pdf", bbox_inches='tight', format='pdf')

In [ ]:
evaluation.compute_and_plot_psd(real_samples_256_raw, gen_samples_uniscale_raw)
plt.savefig(pathfig+"256uniscale_psd.pdf", bbox_inches='tight', format='pdf')

# Gifs

## Real 256 cubes

In [ ]:
# fps=16
# clim = (0,3)
# plot.animate_cubes(real_samples_256, output_name=pathfig+"real256.gif", clim=clim, fps=fps, cmap=plt.cm.plasma)
# animation = plot.cubes_to_animation(real_samples_256, cmap=plt.cm.plasma, clim=clim)
# animation.ipython_display(fps=16, loop=True, autoplay=True)


In [ ]:
# plot.save_animation(real_samples_256[0], gen_samples_256[0], figsize=(10, 6), fps=16, format='mp4', output_file_name=pathfig+'final.mp4', clim=clim)
# plt.style.use('ggplot')

In [ ]:
1